<img src="email_signature_168.png">

***
# Quickdive - Machine Learning (ML)
## München, 2020     
***

Willkommen in **Ihrem persönlichen Jupyter-Notebook**.
  
Sie können in diesem Notebook alle Beispiele live nachvollziehen, aber auch eigene Varianten ausprobieren.  
In der Menüleiste finden sich die wichtigsten Funktionen für "Maus"-User.  
Hier noch einige sehr hilfreiche Tastatur-Kürzel für effizientes Arbeiten mit der Tastatur:

* **Ausführen/Run** einer Zelle mit ... [SHIFT+ENTER]
* Eine neue leere Zelle **über** einer Zelle einfügen mit ... [a] 
* Eine neue leere Zelle **unter** einer Zelle einfügen mit ... [b]
* Eine Zelle **löschen/entfernen** !!VORSICHT!! mit ... [dd]
* Eine Zelle in **Markdown-Format** umwandeln mit ... [m]
* Eine Zelle in **Coding-Format** umwandeln mit ... [y]

Diesen Code müssen wir am Anfang IMMER ausführen:

In [ ]:
%matplotlib inline

# Grundausstattung an Bibliotheken, die wir immer laden
import numpy as np                  # Numerische Operationen, Lineare Algebra
from scipy.stats import *           # Funktionsbibliothek mit statistischen Funktionen
import matplotlib.pyplot as plt     # Funktionsbilio<thek zur Visualisierung von Daten/Ergebnissen
import pandas as pd                 # Bearbeitung von tabellarischen Daten (sog. Data Frames)
import seaborn as sns               # Erweiterte Visualisierung von Daten/Ergebnissen etc.
import warnings                     # Ermöglicht die Deaktivierung von best. Warnmeldungen
import random                       # Damit kann man Zufallszahlen generieren
import os                           # Ermöglicht Zugriff auf das Dateiablagesystem 
import datetime as dt               # Funktionsbiliothek zum Arbeiten mit Zeitreihen Daten
import pickle                       # Ermöglicht das Abspeichern von Objekten (z.B. trainierten Modellen)

# Ein paar Einstellungen, die einem das Leben einfacher machen
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = [8, 4]
plt.style.use('seaborn-white')
from IPython.core.pylabtools import figsize
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
print("Los geht's ...")

# Datenset: BMW-PRICING CHALLENGE

Dafür bearbeiten wir jetzt ein praxisnäheres Beispiel: Das BMW-Pricing Challenge Datenset auf der Plattform KAGGLE  

https://www.kaggle.com/danielkyrka/bmw-pricing-challenge 

Die Autoren dieses Datensets schreiben dazu:

* With this challenge we hope to [...] gain some insight in what the main factors are that drive the value of a used car.  
* The data provided consists of almost 5000 real BMW cars that were sold via a b2b auction in 2018.
* The price shown in the table is the highest bid that was reached during the auction.
* We have also extracted 8 criteria based on the equipment of car that we think might have a good impact on the value of a used car.
* These criteria have been labeled feature1 to feature 8 and are shown in the data below.

In [ ]:
# Zunächst laden wir die Rohdaten
bmw = pd.read_csv("bmw_pricing_challenge.csv")

# Beschränkung auf die 20 am häufigsten vorkommenden Modelle
t20_models = bmw.model_key.value_counts()[:20].index.to_list()  # Auslesen der T20 Modellbezeichnungen
bmw = bmw.loc[bmw.model_key.isin(t20_models),:]  

# Die beiden Datums-Merkmale 'sold_at' und 'registration_date' sollten wir besser in ein Datetime-Format konvertieren
bmw.registration_date = pd.to_datetime(bmw.registration_date)
bmw.sold_at = pd.to_datetime(bmw.sold_at)

# Neue Datums-Features ableiten
bmw["period"] = bmw.sold_at - bmw.registration_date   # erstellt Spalte mit Differenz in Tagen
bmw["period"] = bmw.period.dt.days                    # normiert die Differenz in Tageseinheiten
bmw["Sell_Month"] = bmw.sold_at.dt.month              # Der Monat, in dem die Auktion stattfand

bmw.reset_index(inplace=True)  
bmw.shape
bmw.sample(3)

In [ ]:
# Noch ein kurzer Blick auf die Verteilung der numerischen Variablen ...
bmw.describe().round(1).T

Schauen wir uns noch kurz die Verteilung des Fahrzeugalters an:

In [ ]:
# Verteilung des Fahrzeugalters (in Jahren) im Datenset:
_= (bmw.period/365).hist(bins=70, figsize=(8,5))

Betrachten wir die kategoriellen Features noch etwas genauer: 

In [ ]:
figsize(15,10)
x,y = bmw.model_key, bmw.price
_= sns.boxplot(x, y, data=bmw, color="tomato") 
plt.title("Verteilung der Fahrzeugpreise nach Modellreihen")
plt.xticks(fontsize=14, rotation=80); plt.xlabel("Modellreihe"), plt.ylabel("Preis"); plt.ylim(0,70_000)

plt.show()

Jetzt bauen wir unsere Datenmatrix auf, auf der wir dann das Regressionsmodell trainieren wollen.  

In [ ]:
features = ['model_key', 'mileage', 'engine_power','fuel', 'paint_color', 'car_type',
            'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
            'period', 'Sell_Month', ]


X = bmw[features].copy()
y = bmw.price.copy()

##### (3) & (4) OH-Encoding und Standardisieren

# Wir importieren die Preprocessing Tools aus Scikit-Learn

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler   # Unsere Werkzeuge

# Wir legen ein paar Listen an, um das PreProcessing zu erleichtern
feat_cat = ["model_key", "fuel", "paint_color", "car_type", ] 
feat_num = ['mileage', 'engine_power', 'period',]
feat_bool = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6','feature_7', 'feature_8']
feat_other = ['Sell_Month']

# Jetzt vereinzeln wir die Matrix X in vier Teil-Matrizen 
Xcat   = X[feat_cat].copy()
Xnum   = X[feat_num].copy()
Xbool  = X[feat_bool].copy()
Xother = X[feat_other].copy()

# OH-Encoding auf der Matrix mit den kategoriellen Daten
oh = OneHotEncoder(sparse=False)
Xcat = oh.fit_transform(Xcat)
Xcat_cols = oh.get_feature_names(feat_cat)
Xcat = pd.DataFrame(data=Xcat, columns=Xcat_cols)

# Standardisieren auf der Matrix mit den numerischen Daten
scaler = StandardScaler()
Xnum = scaler.fit_transform(Xnum)
Xnum = pd.DataFrame(Xnum, columns=feat_num)

# Zusammenführen der vier Teilmatrizen zu einer Datenmatrix X
X = pd.concat([Xcat, Xnum, Xbool, Xother], axis=1,  )

print(f"Featurematrix X mit {X.shape[0]} Datensätzen und {X.shape[1]} Feature/Variablen")
print(f"Targetvektor y mit {y.shape[0]} Datensätzen")

In [ ]:
X.head(3)

Trainings- & Testset splitten: Wir splitten in ein Trainingsset mit 70% fürs Training und 30% fürs Testen 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

Lineares Regressionsmodell trainieren 

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)  # Model instanziieren und auf die Trainingsdaten trainieren

scoreTrain = lr.score(X_train, y_train)        # Ermittelt R² Score für Trainingsdaten
scoreTest  = lr.score(X_test, y_test)           # Ermittelt den R² für die Testdaten

print("-"*65)
print(f"Anteil der erklärbaren Varianz, R² auf dem Trainingsset  = {scoreTrain:.2f}")
print(f"Anteil der erklärbaren Varianz, R² auf den TESTDATEN (!) = {scoreTest:.2f}")
print("-"*65)

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor().fit(X_train, y_train)  # Model instanziieren und auf die Trainingsdaten trainieren

scoreTrain = sgd.score(X_train, y_train)        # Ermittelt R² Score für Trainingsdaten
scoreTest  = sgd.score(X_test, y_test)           # Ermittelt den R² für die Testdaten

print("-"*65)
print(f"Anteil der erklärbaren Varianz, R² auf dem Trainingsset  = {scoreTrain:.2f}")
print(f"Anteil der erklärbaren Varianz, R² auf den TESTDATEN (!) = {scoreTest:.2f}")
print("-"*65)

In [ ]:
# Ausgabe der einzelnen Faktoren mit ihren Gewichten in der Regression:
weights = pd.Series(lr.coef_, index=X.columns.to_list(),)
weights.sort_values(ascending=False)[:50]

Mit unserem Modell können wir jetzt den Preis für "neue" ungesehene Daten schätzen:  
Zur Vereinfachung ziehen wir uns aus unseren "unberührten" Testdaten ein Sample und lassen es durch unser Modell schätzen:

In [ ]:
Size = 5
Sample = X_test.sample(Size, random_state=815)
yreal = pd.Series(y_test[Sample.index])
ypred = pd.Series(lr.predict(Sample), index=Sample.index, name="price_pred").astype("int")
result = pd.concat([ypred,yreal,Sample], axis=1)
result.T

# KAGGLE Competition - "Give Me Some Credit"
https://www.kaggle.com/c/GiveMeSomeCredit/data

Das schreiben die Autoren auf KAGGLE:

*Credit scoring algorithms, which make a guess at the probability of default, are the method banks use to determine whether or not a loan should be granted.  
This competition requires participants to improve on the state of the art in credit scoring, by predicting the probability that somebody will experience financial distress in the next two years.*

*The goal of this competition is to build a model that borrowers can use to help make the best financial decisions.*

Hier eine kurze Beschreibung der einzelnen Variablen:

In [ ]:
pd.set_option('display.max_colwidth', -1)
cs_info = pd.read_excel("cs-Data Dictionary.xls", header=1); cs_info

In [ ]:
# Datenset laden
cs = pd.read_csv("cs-training-small.csv")
cs = cs.iloc[:,1:]
cs.info()

Jetzt werfen für mal einen Blick auf die Verteilung der Werte der einzelnen Variablen ...

Zumindest die RUULs liefern einen überdurchschnittlichen Erklärungsbeitrag für unser Modell.  
Wir nehmen die auffälligen Merkmale mit in unsere weiteren Überlegungen.  
Jetzt bauen wir unsere Datenmatrix X und unseren Targetvektor y.

In [ ]:
cs.SeriousDlqin2yrs.value_counts()
cs.SeriousDlqin2yrs.value_counts(normalize=True).round(3)

In [ ]:
X = cs.iloc[:,:-1].copy()
y = cs.iloc[:,-1]
print(X.shape,y.shape)
print(f"Anteil Defaults im gesamten Datenset {y.mean():.3f}")

Wir splitten in ein Trainingsset (2/3) und ein Testset (1/3):

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33333, shuffle=True, stratify=y, random_state=123)
print(f' Trainingsset: {X_train.shape, y_train.shape} / Test Set: {X_test.shape, y_test.shape}')

## Classification mit Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

model = 'Decision Tree'
t_names = ['Kein Default', 'Default']

estimator = DecisionTreeClassifier(class_weight="balanced", ) # max_depth=5
estimator.fit(X_train, y_train)

ytrue = y_test
ypred = estimator.predict(X_test)

accuracy = accuracy_score(ytrue, ypred)
roc_auc = roc_auc_score(ytrue, ypred)
print(f"Dummy-Baseline Accuracy: {1-y_test.mean()}")
print(f'Accuracy Score: {accuracy:.4f}, AUC: {roc_auc:.4f}')
print("\n",classification_report(ytrue, ypred, target_names=t_names))

# Feature Importance aus Model in Dataframe FI schreiben
fi_data = {'Feature': list(X_train.columns), 'F_Importance': estimator.feature_importances_}
FI = pd.DataFrame(data=fi_data)
FI = FI.sort_values('F_Importance', ascending=False); FI

# Confusion Matrix erstellen
mat = confusion_matrix(ytrue, ypred,)
print("Confusion Matrix:\n",mat)

Wir können uns dazu auch eine Confusion-Matrix plotten:

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
category_names = ["Kein Default", "Default"]
sns.heatmap(mat, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=category_names, yticklabels=category_names)
plt.ylabel("Actual")
plt.xlabel("Predicted"); plt.show()

Mit ein paar Optimierungen können wir bereits moderate/gute Ergebnisse erzielen.
Nach diesen ersten "Gehversuchen" schicken wir ein paar weitere Modelle ins Rennen:

## Classification mit verschiedenen Modellen

Beim Decision Tree Classifier ist es nicht notwendig die Daten zu standardisieren.  
Bei den Modellen, die wir jetzt zusätzlich ins Spiel bringen, könnte es sehr hilfreich sein.  
Wir behalten uns diesen Preprocessing-Schritt noch vor und probieren es zunächst ohne Standardisierung.

In [ ]:
# Standardisieren auf der Matrix mit den numerischen Daten
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

Wir bauen uns ein Pipeline aus verschiedenen Classifiern, die wir in einem "Durchgang" auf unsere Trainings- und Testdaten anwenden werden.  
Die einzelnen Schritte:

+ Importieren der notwendigen Classifier Alogrithmen u. verschd. Werkzeuge.
+ Instanziierung der einzelnen Algorithmen (so wird ein konkretes Learner-Objekt daraus).
+ Erstellen einer Pipeline (Festlegen, welche Modelle tatsächlich angewendet werden sollen).
+ Anlegen eines Dataframe, um die Ergebnisse der einzelnen Modelle abzuspeichern.
+ Pipeline-Logik: Ruft die vorab defierten Classifier auf u. wendet sie auf X_train u. X_test an.
+ Ausgeben der Ergebnisse aus unserem Dataframe

In [ ]:
# Importieren der Classifier Algorithmen, die wir als Kandidaten verwenden möchten:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Importieren von Metriken und Zeitfunktionen
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, classification_report
import time

# Hier sind unsere Classifier Kandidaten Modelle
clf1 = GaussianNB()
# clf2 = SVC(class_weight="balanced",)
clf2 = LinearSVC(class_weight="balanced")
clf3 = LogisticRegression(class_weight="balanced")
clf4 = KNeighborsClassifier()
# Dem Random Forest spendieren wir 3 Varianten ...
clf5 = RandomForestClassifier(class_weight="balanced", n_jobs=-1)
clf6 = RandomForestClassifier(n_estimators = 300, class_weight="balanced", max_depth=3,  bootstrap=True, n_jobs=-1)
clf7 = RandomForestClassifier(n_estimators = 500, class_weight="balanced", max_depth=5,  bootstrap=False, n_jobs=-1)

# Das ist unsere Pipeline die wir durchlaufen
pipeline = [(1, "NB",clf1),
           (2, "LinSVM", clf2),
           (3, "LogReg", clf3),
           (4, "Knn5", clf4),
           (5, "RF", clf5),
           (6, "RF opt1", clf6),
           (7, "RF opt2", clf7),
          ]  
# Wir speichern die "Rundenergebnisse" der einzelnen Classifier in einem Dataframe
results = pd.DataFrame( {"Estimator":[], "Accuracy":[], "Precision":[], "Recall":[], "f1":[], "AUC":[], "Duration":[]} )
models_fitted = []  # Ablegen der gefitteten Modelle (Objekte) in einer Liste

# Durchlauf mehrerer Modelle und Wegschreiben des Ergebnisses
for i, name, estimator in pipeline:
    
    # Model fitten u. in Liste ablegen
    print(f"\nFitting {name} ...")
    start = time.time()                     # Stoppuhr: Zwischenzeit nehmen
    est = estimator.fit(X_train, y_train)   # model aus Listing nehmen und fitten
    models_fitted.append(est)

    # Scorings erstellen
    ytrue = y_test                          # ...
    ypred = est.predict(X_test)             # model auf Testdaten anwenden (predict)
    
    acc = accuracy_score(ytrue, ypred )     # Accuracy 
    prec = precision_score(ytrue, ypred )   # Precision 
    rec = recall_score(ytrue, ypred,  )     # Recall
    f1 = f1_score(ytrue, ypred, )           # f1-Score
    auc = roc_auc_score(ytrue, ypred, )     # AUC
    end = time.time()                       # Stoppuhr: Zwischenzeit nehmen
    duration = end - start                  # Walltime in Variable abspeichern
    
    results.loc[i,:] = [name, acc, prec, rec, f1, auc, duration]
    print(f"\nFitting {name} took {duration:.1f} seconds")
    print("-"*100)
    print()
    
print(f"Dummy-Baseline Accuracy: {1-y_test.mean()}")
results.round(3)

In [ ]:
results_not_normalized.round(3)

In [ ]:
results_not_normalized = results.copy()